In [2]:
import geopandas as gpd
from geopandas import GeoDataFrame as GDF

In [3]:
ca_counties = gpd.read_file("examples/sjoin/data/ca-counties.geojson")
ca_counties.head()

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,GlobalID,SHAPE_Length,SHAPE_Area,geometry
0,1,Alameda,ALA,1,01,001,None,{E6F92268-D2DD-4CFB-8B79-5B4B2F07C559},2.538264,0.217411,"MULTIPOLYGON (((-122.27125 37.90503, -122.2702..."
1,2,Alpine,ALP,2,02,003,None,{870479B2-480A-494B-8352-AD60578839C1},2.170420,0.198471,"MULTIPOLYGON (((-119.58667 38.71420, -119.5865..."
2,3,Amador,AMA,3,03,005,None,{4F45B3A6-BE10-461C-8945-6B2AAA7119F6},2.924268,0.161961,"MULTIPOLYGON (((-120.07246 38.70276, -120.0724..."
3,4,Butte,BUT,4,04,007,None,{44FBA680-AECC-4E04-A499-29D69AFFBD4A},4.210365,0.455899,"MULTIPOLYGON (((-121.07661 39.59729, -121.0794..."
4,5,Calaveras,CAL,5,05,009,None,{D11EF739-4A1E-414E-BFD1-E7DCD56CD61E},2.989673,0.275908,"MULTIPOLYGON (((-120.01792 38.43586, -120.0178..."


In [4]:
ca_powerplants = gpd.read_file("examples/sjoin/data/ca-powerplants.geojson")
ca_powerplants.head()

,OBJECTID_1,CECPlantID,PlantName,EIAPlantID,RPS_Eligible,SmallPlant,Peaker,Small_Hydro,SolarThermal,Pumped_Storage,...,Zip,Capacity_Latest,Units,PriEnergySource,LastReportedYear,Lon_WGS84,Lat_WGS84,Annual_Hyperlink,Monthly_Hyperlink,geometry
0,1,B0001,Vaca Dixon Battery Storage,59256,0.0,1.0,0.0,0.0,0.0,0.0,...,95688.0,2.4,Unit 1,BAT,2013.0,-121.921635,38.400098,https://repository.energy.ca.gov/powerplantsge...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-121.92163 38.40010)
1,2,B0002,Tehachapi Storage Project,59661,0.0,1.0,0.0,0.0,0.0,0.0,...,93561.0,NaN,None,None,NaN,-118.380000,35.123334,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-118.38000 35.12333)
2,3,B0003,Yerba Buena Battery,59257,0.0,1.0,0.0,0.0,0.0,0.0,...,95121.0,NaN,None,None,NaN,-121.750114,37.307228,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-121.75011 37.30723)
3,4,B0004,Millikan Avenue BESS,60760,0.0,1.0,0.0,0.0,0.0,0.0,...,92606.0,NaN,None,None,NaN,-117.832291,33.691010,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-117.83229 33.69101)
4,5,B0005,Mira Loma BESS A,60661,0.0,0.0,0.0,0.0,0.0,0.0,...,91761.0,NaN,None,None,NaN,-117.559669,34.004651,https://caenergy.maps.arcgis.com/sharing/rest/...,https://caenergy.maps.arcgis.com/sharing/rest/...,POINT (-117.55967 34.00465)


In [31]:
sjoined = gpd.sjoin(ca_powerplants, ca_counties, how="left", predicate="within")
# sjoined.head()
isinstance(dict(sjoined.dtypes)['geometry'], gpd.array.GeometryDtype)

# sjoined.plot()

True

In [38]:
from typing import Iterable, Optional

def synth_first(synth: Iterable[Optional[object]]) -> object:
    return next(filter(bool, synth))

def print_all_synth(synth: Iterable[Optional[object]]) -> object:
    print('Warning, print_all will take a loooong time')
    any(map(print, filter(bool, synth)))

In [42]:
def synth_sjoin(l: GDF, r: GDF, t: GDF):
    query_preds = l.sindex.valid_query_predicates & r.sindex.valid_query_predicates
    for h in ('left', 'right', 'inner'):
        for p in query_preds:
            res: GDF = gpd.sjoin(l, r, how=h, predicate=p)
            try:
                slim = res[t.columns]
                yield ('sjoin', h, p) if slim.equals(t) else None
            except KeyError:
                yield None


print(synth_first(synth_sjoin(ca_powerplants, ca_counties, sjoined)))

('sjoin', 'left', 'intersects')


In [41]:
def synth_merge(l: GDF, r: GDF, t: GDF):
    'Only trys columns named identically'
    mergeable_columns = l.columns & r.columns
    print(mergeable_columns)
    for h in ('left', 'right', 'inner', 'outer', 'cross'):
        for c in mergeable_columns:
            pass

import inspect as ins
import pandas as pd
print(ins.signature(pd.merge))

print(next(filter(bool, synth_merge(ca_powerplants, ca_counties, None))))

(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'str' = 'inner', on: 'IndexLabel | None' = None, left_on: 'IndexLabel | None' = None, right_on: 'IndexLabel | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool' = True, indicator: 'bool' = False, validate: 'str | None' = None) -> 'DataFrame'
Index(['geometry'], dtype='object')


/var/folders/f3/5cflrcbn48qftqjwwxbn53p00000gn/T/ipykernel_44920/1043050895.py:3: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  mergeable_columns = l.columns & r.columns


TypeError: 'NoneType' object is not iterable